In [1]:
Import-module -ea 'stop' 'ClassExplorer' -pass | Join-String { '{0} = {1}' -f @( $_.Name; $_.Version ) }
| Join-String -op 'Last ran with module[s] versioned: '
$PSVersionTable.PSVersion 
| Join-String -op "Pwsh: "

[Datetime]::now.ToString('u')
| Join-String -op "Date: "
function firstFew { 
    <# 
    .SYNOPSIS
        To simplify output for notebooks, this limits to N items, grabbing from both ends -- or even randomly
    .DESCRIPTION
        Try to be nicer to notebooks, when exceeding limits
    .example
        [Text.StringBuilder]
        | fime
        | firstFew
    #>
    [Alias('firstFew.v0')] # newer version in profile, then in nb module of mine
    [Cmdletbinding()]
    param(
        [int]$Limit = 20,
        [switch]$Shuffle
    )
    $half = $Limit  / 2 

    if( -not $Shuffle) { 
        $Input | Select -first $half -Last $half
        | sort Name -unique
        | ft Name, DisplayString
        return
    }
    $Input | Get-Random -Count $Limit
    | sort Name -unique
    | ft Name, DisplayString
    
}
Set-Alias 'Fime' ClassExplorer\Find-Member -ea ignore # should be defaults


Last ran with module[s] versioned: ClassExplorer = 2.3.3
Pwsh: 7.2.6
Date: 2022-11-12 12:52:52Z


## Find core System span-related types

In [2]:
@( find-type *span* -Namespace System
   find-type *span* -Namespace System.Text
   find-type *span* -Namespace System.Buffers )
| fime
| firstFew

firstFew: 
Line |
   5 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   5 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   5 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   5 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does n

In [3]:
@( find-type *span* -Namespace System
   find-type *span* -Namespace System.Text
   find-type *span* -Namespace System.Buffers )
| sort Namespace 
| % fullname 
# | %{ $_ -replace '^System\.', '' }
# |sort

System.ISpanFormattable
System.ReadOnlySpan`1
System.Span`1
System.TimeSpan
System.ISpanParseable`1
System.Buffers.SpanAction`2
System.Buffers.ReadOnlySpanAction`2
System.Text.SpanLineEnumerator
System.Text.SpanRuneEnumerator


[Custom Query Syntax reference](https://github.com/SeeminglyScience/ClassExplorer/blob/master/docs/en-US/about_Type_Signatures.help.md)

In [4]:
$tinfo_sb = [Text.StringBuilder]
$tinfo_sb
| fime
| firstFew

firstFew: 
Line |
   4 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   4 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   4 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   4 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does n

In [5]:
[Text.StringBuilder]
| Fime


   ReflectedType: System.Text.StringBuilder

Name                  MemberType   Definition
----                  ----------   ----------
Insert                Method       public StringBuilder Insert([0…
Insert                Method       public StringBuilder Insert([0…
Insert                Method       public StringBuilder Insert([0…
Insert                Method       public StringBuilder Insert([0…
Insert                Method       public StringBuilder Insert([0…
Insert                Method       public StringBuilder Insert([0…
Insert                Method       public StringBuilder Insert([0…
AppendFormat          Method       public StringBuilder AppendFormat[9…
AppendFormat          Method       public StringBuilder AppendFormat[9…
AppendFormat          Method       public StringBuilder AppendFormat[9…
AppendFormat          Method       public StringBuilder AppendFormat[9…
AppendFormat          Method       public StringBuilder AppendFormat[9…
AppendFormat         

In [6]:
[System.IO.Stream]
| Find-Member -ParameterType { [anyof[Span[any], Memory[any]]] }
| ft Name, MemberType, DisplayString


Name      MemberType DisplayString
----      ---------- -------------
ReadAsync     Method public virtual ValueTask<int[90…
Read          Method public virtual int Read(Spa…



In [7]:
$inputs = 'bool', 'contains[int]'
@( find-type *span* -Namespace System
   find-type *span* -Namespace System.Text
   find-type *span* -Namespace System.Buffers )
| Join-String -sep ', ' { $_ } -os ']' -op '[anyof['
hr
function new.fime.query.anyof { 
    # this generates the right string
    # the correct string as a variable doesn't work without converting to sb?
    param ( [object[]]$InputTypes )
    $InputTypes
    | Join-String -sep ', ' { $_ } -os ']]' -op '[anyof['
}

new.fime.query.anyof 'int', 'system.boolean'
# Find-Member -ParameterType { [anyof[int, System.Boolean]] }
# | s -First 1

$term = new.fime.query.anyof @('int', 'System.Boolean')

[anyof[System.ISpanFormattable, System.ReadOnlySpan`1[T], System.Span`1[T], timespan, System.ISpanParseable`1[TSelf], System.Text.SpanLineEnumerator, System.Text.SpanRuneEnumerator, System.Buffers.SpanAction`2[T,TArg], System.Buffers.ReadOnlySpanAction`2[T,TArg]]
hr: 
Line |
   6 |  hr
     |  ~~
     | The term 'hr' is not recognized as a name of a cmdlet, function, script file, or executable program.
Check the spelling of the name, or if a path was included, verify that the path is correct and try again.
[anyof[int, system.boolean]]


In [8]:
[Console]
| fime
| firstFew

# $t = [Text.StringBuilder]
# $t | fime
# | select -first 10
# | %{ [pscustomobject]@{ 
#     Name = $_.Name
#     DisplayString = $_.DisplayString.ToString() | oss -Width 999
# } }
# | ft Name, DisplayString
# #| firstFew

firstFew: 
Line |
   3 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   3 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   3 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   3 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does n

In [9]:
'
- Internal class explorer funcs for formatting
- assume they can/will change
'

#$host.ui.RawUI.WindowSize.Width
[ClassExplorer.Internal._Format]
| Fime -Force
| firstFew 




- Internal class explorer funcs for formatting
- assume they can/will change

firstFew: 
Line |
   9 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   9 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   9 |  | firstFew
     |    ~~~~~~~~
     | The input object cannot be bound to any parameters for the command either because the command does not take pipeline input or the input and its properties do not match any of the parameters that take pipeline input.
firstFew: 
Line |
   9 |  | firstFew
     |    ~~~~~~~~
     | The input object cann